# Практическое задание к уроку 1   
## "Введение, примеры задач, бизнес- и ML-метрики"

### Импорт библиотек

In [9]:
import pandas as pd
import numpy as np
import math

# ID товаров

recommended_good_ids = [11, 27, 43, 143, 156, 1134, 991, 1543, 3345, 533] 
bought_good_ids = [32, 143, 521, 991]
grades_good_ids = [1, 2, 3, 1, 1, 2, 3, 1, 2, 1]
prices_recommended_good_ids = [400, 60, 40, 40, 90]
prices_bought_good_ids = [60, 40, 40, 90]

## Расчёт метрик

### Hit Rate

In [10]:
def hit_rate(recommended=[], bought=[]):
    bought = np.array(bought)
    recommended = np.array(recommended)

    intersection = np.isin(bought, recommended)
    hit_rate = (intersection.sum() > 0) * 1

    return hit_rate

def hit_rate_at_k(recommended=[], bought=[], k=5):
    bought = np.array(bought)
    recommended = np.array(recommended[:k])

    intersection = np.isin(bought, recommended)
    hit_rate = (intersection.sum() > 0) * 1

    return hit_rate

In [11]:
hit_rate_at_k(recommended_good_ids, bought_good_ids, k=5)

1

## Precision & Money Precision

In [12]:
def precision(recommended, bought):
    bought = np.array(bought)
    recommended = np.array(recommended)

    intersection = np.isin(bought, recommended)
    precision = intersection.sum() / len(recommended)

    return precision


def precision_at_k(recommended, bought, k=5):
    bought = np.array(bought)
    recommended = np.array(recommended)
    recommended = recommended[:k]

    intersection = np.isin(bought, recommended)
    precision = intersection.sum() / len(recommended)


    return precision


def money_precision_at_k(recommended, bought, prices_recommended, k=5):
    recommend = np.array(recommended)[:k]
    prices_recommended = np.array(prices_recommended)[:k]

    intersection = np.isin(recommend, bought)
    precision = np.dot(intersection, prices_recommended) / prices_recommended.sum()

    return precision

In [13]:
money_precision_at_k(recommended_good_ids, bought_good_ids, prices_recommended_good_ids, k=5)

0.06349206349206349

## Recall & Money Recall

In [17]:
def recall(recommended, bought):
    bought = np.array(bought)
    recommended = np.array(recommended)
    
    intersection = np.isin(bought, recommended)
    recall = intersection.sum() / len(bought)

    return recall


def recall_at_k(recommended, bought, k=5):
    bought = np.array(bought)
    recommended = np.array(recommended)
    recommended = recommended[:k]

    intersection = np.isin(bought, recommended)
    recall = intersection.sum() / len(bought)

    return recall


def money_recall_at_k(recommended, bought, prices_recommended, prices_bought, k=5):
    bought = np.array(bought)
    prices_bought = np.array(prices_bought)
    recommended = np.array(recommended)[:k]
    prices_recommended = np.array(prices_recommended)[:k]

    intersection = np.isin(recommended, bought)
    recall = np.dot(intersection, prices_recommended) / prices_bought.sum()

    return recall

In [20]:
recall_at_k(recommended_good_ids, bought_good_ids, 5)

0.25

In [24]:
money_recall_at_k(recommended_good_ids, bought_good_ids, prices_recommended_good_ids, prices_bought_good_ids, 5)

0.17391304347826086

## Mean Average Precision at k (MAP@k)

In [26]:
def ap_k(recommended, bought, k=5):
    bought = np.array(bought)
    recommended = np.array(recommended)

    intersection = np.isin(recommended, bought)

    if sum(intersection) == 0:
        return 0

    sum_ = 0
    for i in range(0, k-1):
        if intersection[i] == True:
            p_k = precision_at_k(recommended, bought, k=i+1)
            sum_ += p_k

    result = sum_ / sum(intersection)

    return result

def map_k(recommended_all_users, bought_all_users, k=5):
    sum_ap_k = sum([ap_k(recommended_all_users[u], bought_all_users[u], k) for u in range(len(bought_all_users))])
    map_k = sum_ap_k / len(bought_all_users)

    return map_k

In [27]:
map_k([recommended_good_ids for i in range(5)], [bought_good_ids[:i] for i in range(5)], k=5)

0.125

## Mean Reciprocal Rank at k (MRR@k)

In [29]:
def reciprocal_rank(recommended_all_users, bought_all_users, bought, k=5):
    recommended_all_users = np.array(recommended_all_users)
    bought_all_users = np.array(bought_all_users)

    rr_list = []
    for i in np.arange(len(bought)):
        relevant_indexes = np.nonzero(np.isin(recommended_all_users[i][:k], bought_all_users[i]))[0]
        if len(relevant_indexes) != 0:
            rr_list.append(1/(relevant_indexes[0]+1))


    return sum(rr_list)/len(bought)

In [31]:
reciprocal_rank([recommended_good_ids for i in range(5)], [bought_good_ids[:i] for i in range(5)], bought_good_ids, k=5)

<ipython-input-29-4b121ef60bbf>:3: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  bought_all_users = np.array(bought_all_users)


0.125

## NDCG@k

In [32]:
grades_list = [3, 2, 1, 1, 3, 1, 2]

In [34]:
def dcg_at_k(grades_list, k=7):
    grades_list = np.array(grades_list)[:k]

    dcg = 0
    for i in range(len(grades_list)):
        if i == 0:
            dcg = grades_list[i]
        else:
            dcg += (grades_list[i] / math.log2(i+1+1))

    return dcg

In [36]:
dcg_at_k(grades_list, k=7), dcg_at_k(sorted(grades_list, reverse=True), k=7)

(7.375968340694622, 7.830535704537055)

In [38]:
def ndcg_at_k(grades_list, k=7):
    dcg = dcg_at_k(grades_list, k)
    ideal_dcg = dcg_at_k(sorted(grades_list, reverse=True), k)

    ndcg = dcg / ideal_dcg

    return ndcg

In [39]:
ndcg_at_k(grades_list, k=7)

0.9419493913323128